In [12]:
from data.prepareData import fetch_sarif_dataframes
import pandas as pd
import os

# Define the input variables
asd = "asd"
REPO_OWNER = "alaynfm"
REPO_NAME = "techFriday"
WORKFLOW_ID = "comparative.yaml"

sarif_dataframes_by_tool = fetch_sarif_dataframes(asd, REPO_OWNER, REPO_NAME, WORKFLOW_ID)

None


In [4]:
sarif_dataframes_by_tool_cleaned = {key.lower(): df.drop_duplicates() for key, df in sarif_dataframes_by_tool.items()}
sarif_dataframes_by_tool = {key.lower(): df for key, df in sarif_dataframes_by_tool.items()}

In [5]:
# Initialize a list to store the comparison data
comparison_data = {
    "Tool": [],
    "AWS Controls": [],
    "Azure Controls": [],
    "Total Controls": []
}

# Iterate through all tools in sarif_dataframes_by_tool
for tool_name, df in sarif_dataframes_by_tool_cleaned.items():
    aws_count = df[df["source"] == "AWS"].shape[0]
    azure_count = df[df["source"] == "Azure"].shape[0]
    total_count = df.shape[0]
    
    comparison_data["Tool"].append(tool_name)
    comparison_data["AWS Controls"].append(aws_count)
    comparison_data["Azure Controls"].append(azure_count)
    comparison_data["Total Controls"].append(total_count)

# Convert the comparison data to a DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Display the DataFrame
comparison_df

,Tool,AWS Controls,Azure Controls,Total Controls


In [6]:
# Initialize a list to store the comparison data
comparison_data2 = {
    "Tool": [],
    "AWS Controls": [],
    "Azure Controls": [],
    "Total Controls": []
}

# Iterate through all tools in sarif_dataframes_by_tool
for tool_name, df in sarif_dataframes_by_tool.items():
    aws_count = df[df["source"] == "AWS"].shape[0]
    azure_count = df[df["source"] == "Azure"].shape[0]
    total_count = df.shape[0]
    
    comparison_data2["Tool"].append(tool_name)
    comparison_data2["AWS Controls"].append(aws_count)
    comparison_data2["Azure Controls"].append(azure_count)
    comparison_data2["Total Controls"].append(total_count)

# Convert the comparison data to a DataFrame
comparison_data2 = pd.DataFrame(comparison_data2)

# Display the DataFrame
comparison_data2

,Tool,AWS Controls,Azure Controls,Total Controls


In [7]:
print(198-104)
269 - 178

94


91